In [39]:
import sys
sys.path.append('..')
import operations, models
from operations.database import Database

# from queries import *
import json
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import psycopg2

In [40]:
uniqueID= '04352254C'

In [41]:
db=Database()
x = db.read_from_db("SELECT uniqueID, description, title from product_table;")
df = pd.DataFrame(x, columns=['uniqueID','Description', 'Title'])
df_feature_matrix = df['Title'].str.lower() + ' ' + df['Description'].str.lower()


In [42]:
df

,uniqueID,Description,Title
0,01705319,"Our performance dress shirt is easy care, so t...",slim button-down wrinkle resistant performance...
1,07207835C,Create your sexiest silhouette with our curve-...,mid rise medium wash skinny jeans
2,09725333,Add chic style to your layered looks with this...,stripe duster
3,07199265,"Fierce and flattering, this stretch-infused je...",mid rise leopard print jean ankle leggings
4,06507065,We love the looks that you can create with thi...,pleated balloon sleeve thong bodysuit
...,...,...,...
2995,09785587C,Stand out from the crowd in this top that feat...,bell sleeve metallic top
2996,00283671,"An essential in every wardrobe, this shirt loo...",classic soft wash white oxford shirt
2997,80002561,Inspired by days spent in the sun by the water...,breda silver jane watch
2998,80001472,"For a sun-kissed look all year round, choose o...",carter beauty mini bronzer palette


In [43]:
df_feature_matrix

0       slim button-down wrinkle resistant performance...
1       mid rise medium wash skinny jeans create your ...
2       stripe duster add chic style to your layered l...
3       mid rise leopard print jean ankle leggings fie...
4       pleated balloon sleeve thong bodysuit we love ...
                              ...                        
2995    bell sleeve metallic top stand out from the cr...
2996    classic soft wash white oxford shirt an essent...
2997    breda silver jane watch inspired by days spent...
2998    carter beauty mini bronzer palette for a sun-k...
2999    machete light tortoise grande drop earrings so...
Length: 3000, dtype: object

In [44]:
vectorizer = CountVectorizer(stop_words='english')
vectorized = vectorizer.fit_transform(df_feature_matrix)


In [45]:
similarities = cosine_similarity(vectorized)
df_processed = pd.DataFrame(similarities, columns=df['Title']+' '+ df['Description'], index=df['Title']+' '+ df['Description']).reset_index()

In [46]:
df_processed

,index,"slim button-down wrinkle resistant performance dress shirt Our performance dress shirt is easy care, so that you can look sharp with little effort. Wear this shirt from work days to weekends with chinos or jeans.","mid rise medium wash skinny jeans Create your sexiest silhouette with our curve-hugging skinny jean. With an alluring mid rise and a classic wash, your legs never had it so good.","stripe duster Add chic style to your layered looks with this supersoft duster, designed with a lightweight drape and fresh stripes.","mid rise leopard print jean ankle leggings Fierce and flattering, this stretch-infused jean promises comfort, style and an eye-catching outfit thanks to the bold leopard print. Cut at ankle length with a mid rise fit.",pleated balloon sleeve thong bodysuit We love the looks that you can create with this long sleeve bodysuit. It stays perfectly tucked in under your favorite jeans or a high waisted skirt. Great for versatile styling.,"extra slim light gray luxe comfort knit suit jacket Tailored for an extra slim fit and sharp look, this wrinkle-resistant jacket keeps its shape all-night long with just the amount of stretch. Pair with one of our dress shirts and the matching pant for your next big meeting or event.","emory park floral print cut-out back mini dress This floral print mini dress is perfect for all your weekend plans, featuring a ruffle hem and cut-out back. Pair with open toe heels.",steve madden malibu genuine hair pumps Leopard spotted pony hair puts a fun feline twist on the super-sexy Malibu pump from featured brand Steve Madden. A partially transparent upper lends this stiletto a modern element while revealing just the right amount of skin.,melissa black pointy iii flat The Melissa Pointy III features a resin buckle with bow detailing and a pointed toe.,...,"steve madden grate platform booties Step out with '70s-style swagger donning the Grate bootie from Steve Madden. A platform sole and subtly flared block heel give this leather looker retro proportions, while an of-the-moment cropped shaft ensures versatility.","gold tortoiseshell square rim sunglasses These chic sunglasses protect your peepers with 100% UV protection, while square frames with a tortoiseshell pattern keep your look in the now.","amped & co green 'cactus' neon light Real neon, handblown glass in the shape of a Cactus. Neon glows a memorizing green color when turned on. Mounted on a black base with an anti-slip pad. Perfect for home decor, the Cactus Neon Light is a unique decorative room light.","slim medium wash stretch+ soft cotton jeans Made from soft fabric for all-day comfort, this medium wash take on our slim fit is an ideal choice when you want something to wear for a casual day at the office straight into date night.","neon heart print performance sport trunks Shopping for Valentine's Day. These are a great pair to gift or for spicing up your own underwear drawer. Our performance sport trunks promise cool comfort thanks to moisture-wicking, 4-way stretch fabric.","bell sleeve metallic top Stand out from the crowd in this top that features metallic fabric, a lightweight drape and flowy bell sleeves.","classic soft wash white oxford shirt An essential in every wardrobe, this shirt looks dapper at work or for play. A solid color and classic, tailored design pairs easily with jeans or shorts for a versatile piece. The soft wash process means you'll want to live in it.","breda silver jane watch Inspired by days spent in the sun by the water and followed by evenings of indulgence. The Jane carries a delicate 23mm oval case, champagne colored mother of pearl dial and stainless steel bracelet that fastens with a jewelry clasp.","carter beauty mini bronzer palette For a sun-kissed look all year round, choose our Bronzer Palette. A mix of matte and shimmering shades of brown add instant colour and depth to the high points of the face.","machete light tortoise grande drop earrings Sophisticated & playful patterns ador

In [47]:
desc_dyn=df[df['uniqueID'] == str(uniqueID)]['Description']
title_dyn=df[df['uniqueID'] == str(uniqueID)]['Title']

In [48]:
input_prod= str(title_dyn.iloc[-1]) +' '+  desc_dyn.iloc[-1] 
print(input_prod)
# print(df_processed.nlargest(5,input_prod).iloc[1:1,:])
sim_score = pd.DataFrame(df_processed.nlargest(5,input_prod)[input_prod])
recommendations = pd.DataFrame(df_processed.nlargest(5,input_prod)['index'])
recommendations = recommendations[recommendations['index']!=input_prod]
sim_score.columns=['Score']
# print(recommendations)

classic dusty blue wool-blend wrinkle-resistant performance suit jacket Our classic fit suit jacket made from a breakthrough stretch fabric that moves with you and keeps its shape. Just pair it with the matching pant and a dress shirt, and you'll be ready for anything.


In [49]:
recommendations

,index
1762,classic black performance stretch wool-blend s...
1979,classic dusty blue wool-blend wrinkle-resistan...
1460,classic charcoal wool blend wrinkle-resistant ...
2986,slim charcoal wool blend wrinkle-resistant per...


In [50]:
temp = pd.concat([df.iloc[recommendations.index], sim_score], axis=1)

In [51]:
y = db.read_from_db("SELECT uniqueId, price, title, image_url from product_table where uniqueId in %s", (tuple(temp['uniqueID'].dropna()), ))
y

[('04352239',
  '298',
  'classic charcoal wool blend wrinkle-resistant performance suit jacket',
  'https://images.express.com/is/image/expressfashion/0039_04352239_0904?cache=on&wid=361&fmt=jpeg&qlt=75,1&resmode=sharp2&op_usm=1,1,5,0&defaultImage=Photo-Coming-Soon'),
 ('04351814C',
  '248',
  'classic black performance stretch wool-blend suit jacket',
  'https://images.express.com/is/image/expressfashion/0039_04351814_0098?cache=on&wid=361&fmt=jpeg&qlt=75,1&resmode=sharp2&op_usm=1,1,5,0&defaultImage=Photo-Coming-Soon'),
 ('03252255C',
  '128',
  'classic dusty blue wool-blend wrinkle-resistant performance suit pant',
  'https://images.express.com/is/image/expressfashion/0039_03252255_1336?cache=on&wid=361&fmt=jpeg&qlt=75,1&resmode=sharp2&op_usm=1,1,5,0&defaultImage=Photo-Coming-Soon'),
 ('04352237',
  '298',
  'slim charcoal wool blend wrinkle-resistant performance suit jacket',
  'https://images.express.com/is/image/expressfashion/0039_04352237_0904?cache=on&wid=361&fmt=jpeg&qlt=75,